# 利用逻辑回归对癌症进行预测

## 步骤
- 1、读取数据
- - 读取的时候加上names
- 2、数据处理
- - 处理缺失值
- 3、数据集划分
- 4、特征工程
- - 无量纲化处理
- 5、逻辑回归预估器
- 6、模型评估

In [41]:
import pandas as pd
import numpy as np

# 1、读取数据
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
column_names = [
    'Sample code number',
    'Clump Thickness',
    'Uniformity of Cell Size',
    'Uniformity of Cell Shape',
    'Marginal Adhesion',
    'Single Epithelial Cell Size',
    'Bare Nuclei',
    'Bland Chromatin',
    'Normal Nucleoli',
    'Mitoses',
    'Class'
]
data = pd.read_csv(path, names=column_names)

In [42]:
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [43]:
# 2、缺失值处理
data = data.replace(to_replace="?", value=np.nan)
# 删除缺失样本
data.dropna(inplace=True)

In [44]:
data.isnull().any() # 不存在缺失值了

Sample code number             False
Clump Thickness                False
Uniformity of Cell Size        False
Uniformity of Cell Shape       False
Marginal Adhesion              False
Single Epithelial Cell Size    False
Bare Nuclei                    False
Bland Chromatin                False
Normal Nucleoli                False
Mitoses                        False
Class                          False
dtype: bool

In [45]:
# 3、划分数据集
from sklearn.model_selection import train_test_split
# 筛选特征值和目标值
x = data.iloc[:, 1:-1]
y = data["Class"]
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [46]:
# 4、特征工程
# 标准化
from sklearn.preprocessing import StandardScaler
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [47]:
# 5、逻辑回归预估器
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()
estimator.fit(x_train, y_train)

LogisticRegression()

In [48]:
# 逻辑回归的模型参数：回归系数和偏置
# 回归系数
estimator.coef_

array([[1.19777169, 0.10877967, 0.73209957, 0.60323232, 0.12122898,
        1.48162508, 0.75112762, 0.79980762, 0.82133788]])

In [49]:
# 偏置
estimator.intercept_

array([-1.0749105])

In [50]:
# 6、模型评估
# 方法1：直接比对真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("直接比对真实值和预测值：\n", y_test == y_predict)
# 方法2：计算准确率
score = estimator.score(x_test, y_test)
print("准确率为：\n", score)

y_predict:
 [2 4 4 2 2 2 2 2 2 2 2 2 2 4 2 2 4 4 4 2 4 2 4 4 4 2 4 2 2 2 2 2 4 2 2 2 4
 2 2 2 2 4 2 4 4 4 4 2 4 4 2 2 2 2 2 4 2 2 2 2 4 4 4 4 2 4 2 2 4 2 2 2 2 4
 2 2 2 2 2 2 4 4 4 2 4 4 4 4 2 2 2 4 2 4 2 2 2 2 2 2 4 2 2 4 2 2 4 2 4 4 2
 2 2 2 4 2 2 2 2 2 2 4 2 4 2 2 2 4 2 4 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 4 2 4
 2 2 4 4 4 2 2 4 4 2 4 4 2 2 2 2 2 4 4 2 2 2 4]
直接比对真实值和预测值：
 389     True
32      True
272     True
655     True
271     True
       ...  
296    False
585     True
256     True
43     False
691     True
Name: Class, Length: 171, dtype: bool
准确率为：
 0.9766081871345029


In [51]:
# 查看精确率、召回率、F1-score
from sklearn.metrics import classification_report

report = classification_report(y_test, y_predict, labels=[2, 4], target_names=['良性', '恶性'])
print(report)

              precision    recall  f1-score   support

          良性       0.98      0.98      0.98       111
          恶性       0.97      0.97      0.97        60

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



In [52]:
# y_true：每个样本的真实类别，必须为 0 （反例），1 （正例）标记
# 将y_test转换成 0 1
y_true = np.where(y_test > 3, 1, 0)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_predict)

0.9743243243243243